# Day 11
https://adventofcode.com/2018/day/11

In [1]:
import aocd
data = aocd.get_data(year=2018, day=11)

In [2]:
def hundreds(n):
    return int(str(n).zfill(3)[-3])

In [3]:
def power_level(x, y, grid_serial):
    rack = x + 10
    power = ((rack * y) + grid_serial) * rack
    return hundreds(power) - 5

In [4]:
examples = (
    ((3, 5, 8), 4),
    ((122, 79, 57), -5),
    ((217, 196, 39), 0),
    ((101, 153, 71), 4),
)
for (args, expected) in examples:
    print('power_level{} == {} : {}'.format(args, expected, power_level(*args) == expected))

power_level(3, 5, 8) == 4 : True
power_level(122, 79, 57) == -5 : True
power_level(217, 196, 39) == 0 : True
power_level(101, 153, 71) == 4 : True


##### Trickery!
To get through part 2 in particular without running calculations like mad, we need to create this summed area table: we iterate through each potential *bottom corner* for squares and calculate them all at once.

In [5]:
def grid_power_squares(serial):
    sat = dict()
    for x in range(1, 301):
        for y in range(1, 301):
            p1 = power_level(x, y, serial)
            sat[(x, y)] = sum((
                p1,
                sat.get((x, y-1), 0),
                sat.get((x-1, y), 0),
                -sat.get((x-1, y-1), 0)
            ))
            for squaresize in range(1, min(x, y)):
                corners = (
                    sat.get((x, y), 0),
                    sat.get((x-squaresize, y-squaresize), 0),
                    -sat.get((x, y-squaresize), 0),
                    -sat.get((x-squaresize, y), 0),
                )
                yield ((x-squaresize+1, y-squaresize+1, squaresize), sum(corners))            

In [6]:
def best_grid_power_squares(serial):
    bestsquares = dict()
    bestvalues = dict()
    for ((x, y, squaresize), value) in grid_power_squares(serial):
        if value > bestvalues.get(squaresize, 0):
            bestsquares[squaresize] = (x, y, squaresize)
            bestvalues[squaresize] = value
        if value > bestvalues.get('all', 0):
            bestsquares['all'] = (x, y, squaresize)
            bestvalues['all'] = value
    return bestsquares

In [7]:
serial = int(data)
bgps = best_grid_power_squares(serial)
p1 = bgps.get(3)[:2]
print('Part 1: {},{}'.format(*p1))
p2 = bgps.get('all')
print('Part 2: {},{},{}'.format(*p2))

Part 1: 20,68
Part 2: 231,273,16
